# 1. 라이브러리 임포트

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np

import json
import os
from statistics import median

from PIL import Image, ImageOps, ImageDraw
import cv2

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50V2, ResNet101V2, ResNet152V2, ConvNeXtTiny, ConvNeXtSmall, ConvNeXtBase
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, LayerNormalization, Dense

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
!pip install koreanize-matplotlib
import koreanize_matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 34.0 MB/s eta 0:00:00


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# 2. 데이터 불러오기 및 데이터프레임 생성하기

In [ ]:
# json path 생성
label_path = '/content/drive/MyDrive/Data/label'
train_label_path = os.path.join(label_path, 'train')
val_label_path = os.path.join(label_path, 'val')
label_json_list = ['anger.json', 'happy.json', 'panic.json', 'sadness.json']

# image path 생성
image_path = '/content/drive/MyDrive/preprocessed_crop'
train_path = os.path.join(image_path,'train')
val_path = os.path.join(image_path,'val')

# segmentation path 생성
seg_path = '/content/drive/MyDrive/preprocessed_seg'
train_seg_path = os.path.join(seg_path,'train')
val_seg_path = os.path.join(seg_path,'val')

In [ ]:
# train image list 생성
train_path_anger = os.path.join(train_path, 'anger')
train_path_happy = os.path.join(train_path, 'happy')
train_path_panic = os.path.join(train_path, 'panic')
train_path_sadness = os.path.join(train_path, 'sadness')

train_path_anger = os.listdir(train_path_anger)
train_path_happy = os.listdir(train_path_happy)
train_path_panic = os.listdir(train_path_panic)
train_path_sadness = os.listdir(train_path_sadness)

# val image path list 생성
val_path_anger = os.path.join(val_path, 'anger')
val_path_happy = os.path.join(val_path, 'happy')
val_path_panic = os.path.join(val_path, 'panic')
val_path_sadness = os.path.join(val_path, 'sadness')

val_path_anger = os.listdir(val_path_anger)
val_path_happy = os.listdir(val_path_happy)
val_path_panic = os.listdir(val_path_panic)
val_path_sadness = os.listdir(val_path_sadness)

In [ ]:
# train image_seg list 생성
train_seg_path_anger = os.path.join(train_seg_path, 'anger')
train_seg_path_happy = os.path.join(train_seg_path, 'happy')
train_seg_path_panic = os.path.join(train_seg_path, 'panic')
train_seg_path_sadness = os.path.join(train_seg_path, 'sadness')

train_seg_path_anger = os.listdir(train_seg_path_anger)
train_seg_path_happy = os.listdir(train_seg_path_happy)
train_seg_path_panic = os.listdir(train_seg_path_panic)
train_seg_path_sadness = os.listdir(train_seg_path_sadness)

val_seg_path_anger = os.path.join(val_seg_path, 'anger')
val_seg_path_happy = os.path.join(val_seg_path, 'happy')
val_seg_path_panic = os.path.join(val_seg_path, 'panic')
val_seg_path_sadness = os.path.join(val_seg_path, 'sadness')

val_seg_path_anger = os.listdir(val_seg_path_anger)
val_seg_path_happy = os.listdir(val_seg_path_happy)
val_seg_path_panic = os.listdir(val_seg_path_panic)
val_seg_path_sadness = os.listdir(val_seg_path_sadness)

In [ ]:
# image 개수 확인(happy, panic의 개수가 줄어든 이유: 중복된 데이터가 존재했음)
print(len(list(train_path_anger)), len(list(train_path_happy)), len(list(train_path_panic)), len(list(train_path_sadness)))
print(len(list(val_path_anger)), len(list(val_path_happy)), len(list(val_path_panic)), len(list(val_path_sadness)))

# image_seg 개수 확인
print(len(list(train_seg_path_anger)), len(list(train_seg_path_happy)), len(list(train_seg_path_panic)), len(list(train_seg_path_sadness)))
print(len(list(val_seg_path_anger)), len(list(val_seg_path_happy)), len(list(val_seg_path_panic)), len(list(val_seg_path_sadness)))

1500 1494 1500 1500
300 300 300 300
1500 1494 1500 1500
300 300 300 300


In [ ]:
# DataFrame column 생성
train_df = pd.DataFrame(columns=['img_path','age', 'gender', 'isProf', 'background', 'maxX','maxY','minX','minY','label'])
val_df = pd.DataFrame(columns=['img_path','age', 'gender', 'isProf', 'background', 'maxX','maxY','minX','minY','label'])

train_seg_df = pd.DataFrame(columns=['img_path','age', 'gender', 'isProf', 'background', 'maxX','maxY','minX','minY','label'])
val_seg_df = pd.DataFrame(columns=['img_path','age', 'gender', 'isProf', 'background', 'maxX','maxY','minX','minY','label'])

In [ ]:
import json
def makedf(dir_path, df, data_path):
    i = 0
    data_name = dir_path.split('/')[-1]
    for label_name in label_json_list:
        with open(os.path.join(dir_path, data_name + '_' + label_name), 'r', encoding='cp949') as f:
            file = json.load(f)
        for v in file:
            if v['faceExp_uploader'] == '분노':
                label = 'anger'
            elif v['faceExp_uploader'] == '기쁨':
                label = 'happy'
            elif v['faceExp_uploader'] == '당황':
                label = 'panic'
            elif v['faceExp_uploader'] == '슬픔':
                label = 'sadness'
            minX = median([v['annot_A']['boxes']['minX'], v['annot_B']['boxes']['minX'], v['annot_C']['boxes']['minX']])
            minY = median([v['annot_A']['boxes']['minY'], v['annot_B']['boxes']['minY'], v['annot_C']['boxes']['minY']])
            maxX = median([v['annot_A']['boxes']['maxX'], v['annot_B']['boxes']['maxX'], v['annot_C']['boxes']['maxX']])
            maxY = median([v['annot_A']['boxes']['maxY'], v['annot_B']['boxes']['maxY'], v['annot_C']['boxes']['maxY']])
            result_path = os.path.join(data_path, label)
            df.loc[i] = [os.path.join(result_path, v['filename']), v['age'], v['gender'], v['isProf'], v['bg_uploader'], int(maxX), int(maxY), int(minX), int(minY), label]
            i += 1

    return df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# DataFrame 생성
train_df = makedf(train_label_path, train_df, train_path)
val_df = makedf(val_label_path, val_df, val_path)

train_seg_df = makedf(train_label_path, train_seg_df, train_seg_path)
val_seg_df = makedf(val_label_path, val_seg_df, val_seg_path)

# 3. Preprocessing

## 3-1. crop + segment된 사진 정보를 이용하여 좌표정보 확인(5개 데이터에서 음수값 확인)

In [ ]:
train_df[(train_df['minX'] < 0) | (train_df['minY'] < 0) | (train_df['maxX'] < 100) | (train_df['maxY'] < 130)]

,img_path,age,gender,isProf,background,maxX,maxY,minX,minY,label
316,/content/drive/MyDrive/preprocessed_crop/train...,40,여,일반인,공공시설/종교/의료시설,863,1585,-27,462,panic
1027,/content/drive/MyDrive/preprocessed_crop/train...,20,남,일반인,도심 환경,1582,709,1001,-81,anger
3837,/content/drive/MyDrive/preprocessed_crop/train...,20,남,일반인,공공시설/종교/의료시설,965,857,261,-20,happy
5123,/content/drive/MyDrive/preprocessed_crop/train...,20,여,일반인,상업시설/점포/시장,1493,672,870,-204,anger
5270,/content/drive/MyDrive/preprocessed_crop/train...,30,여,일반인,공공시설/종교/의료시설,361,386,-2,-102,sadness


## 3-2. 음수값을 0으로 변환

In [ ]:
if (train_df['minX'] < 0).any():
    train_df.loc[train_df['minX'] < 0, 'minX'] = 0

if (train_df['minY'] < 0).any():
    train_df.loc[train_df['minY'] < 0, 'minY'] = 0

In [ ]:
train_df[(train_df['minX'] < 0) | (train_df['minY'] < 0) | (train_df['maxX'] < 100) | (train_df['maxY'] < 130)]

,img_path,age,gender,isProf,background,maxX,maxY,minX,minY,label


In [ ]:
val_df[(val_df['minX'] < 0) | (val_df['minY'] < 0) | (val_df['maxX'] < 100) | (val_df['maxY'] < 130)]

,img_path,age,gender,isProf,background,maxX,maxY,minX,minY,label


## 3-3. segmentation 사진 데이터를 이용하여 for 문을 통해 mean값이 10보다 작은 사진 데이터 탐지

이유: 검정색이 대부분인 데이터 즉, 사진 crop이 이상하게 된 데이터를 탐지하여 제거하기 위함

In [ ]:
for i in range(len(train_seg_df)):
    img = cv2.imread(train_seg_df['img_path'][i])
    mean_val1 = np.mean(img, axis=0)
    mean_val2 = np.mean(mean_val1)
    if mean_val2 < 10:
	    print(train_seg_df['img_path'][i], f'index: {i}')

/content/drive/MyDrive/preprocessed/train/happy/k3s935e50e666fee414ce63c6a7cc0563dbee792014b083a74cbcbf602b735rqt.jpg index: 889
/content/drive/MyDrive/preprocessed/train/sadness/vag7dfda5e3b9561fd12c1175d277b1d2c1f22dd2a5d37b3286dd4b3e2d45pili.jpg index: 5270


In [ ]:
for i in range(len(val_seg_df)):
    img = cv2.imread(val_seg_df['img_path'][i])
    mean_val1 = np.mean(img, axis=0)
    mean_val2 = np.mean(mean_val1)
    if mean_val2 < 10:
	    print(val_seg_df['img_path'][i], f'index: {i}')

/content/drive/MyDrive/preprocessed/val/sadness/rb04c532132d836a981bf48f70b3588730632285f2c97b347e1cdb4fa1133mcty.jpg index: 566


## 3-4. 사진 정보 확인 후 drop을 통해 제거

### 3-4-1. train 데이터 제거

In [ ]:
train_df[train_df['img_path'] == '/content/drive/MyDrive/preprocessed_crop/train/happy/k3s935e50e666fee414ce63c6a7cc0563dbee792014b083a74cbcbf602b735rqt.jpg']

,img_path,age,gender,isProf,background,maxX,maxY,minX,minY,label
889,/content/drive/MyDrive/preprocessed_crop/train...,40,남,일반인,실외 자연환경,1004,1691,0,0,happy


In [ ]:
train_df[train_df['img_path'] == '/content/drive/MyDrive/preprocessed_crop/train/sadness/vag7dfda5e3b9561fd12c1175d277b1d2c1f22dd2a5d37b3286dd4b3e2d45pili.jpg']

,img_path,age,gender,isProf,background,maxX,maxY,minX,minY,label
5270,/content/drive/MyDrive/preprocessed_crop/train...,30,여,일반인,공공시설/종교/의료시설,361,386,0,0,sadness


In [ ]:
train_df = train_df.drop(index=[889, 5270])

In [ ]:
train_df[train_df['img_path'] == '/content/drive/MyDrive/preprocessed_crop/train/happy/k3s935e50e666fee414ce63c6a7cc0563dbee792014b083a74cbcbf602b735rqt.jpg']

,img_path,age,gender,isProf,background,maxX,maxY,minX,minY,label


In [ ]:
train_df[train_df['img_path'] == '/content/drive/MyDrive/preprocessed_crop/train/sadness/vag7dfda5e3b9561fd12c1175d277b1d2c1f22dd2a5d37b3286dd4b3e2d45pili.jpg']

,img_path,age,gender,isProf,background,maxX,maxY,minX,minY,label


### 3-4-2. val 데이터 제거

In [ ]:
val_df[val_df['img_path'] == '/content/drive/MyDrive/preprocessed_crop/val/sadness/rb04c532132d836a981bf48f70b3588730632285f2c97b347e1cdb4fa1133mcty.jpg']

,img_path,age,gender,isProf,background,maxX,maxY,minX,minY,label
566,/content/drive/MyDrive/preprocessed_crop/val/s...,50,여,일반인,숙박 및 거주공간,2543,1342,1689,243,sadness


In [ ]:
val_df = val_df.drop(index=[566])

In [ ]:
val_df[val_df['img_path'] == '/content/drive/MyDrive/preprocessed_crop/val/sadness/rb04c532132d836a981bf48f70b3588730632285f2c97b347e1cdb4fa1133mcty.jpg']

,img_path,age,gender,isProf,background,maxX,maxY,minX,minY,label


## 3-5. 사진 데이터 array 변환, 차원 추가, resize 전처리 작업 수행

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array

norm_layer = tf.keras.layers.Normalization(
    mean = [0.485 * 255, 0.456 * 255, 0.406 * 255],
    variance = [(0.229 * 255) ** 2, (0.224 * 255) ** 2, (0.225 * 255) ** 2],
)
def preprocess_image(image,size=224):
    image = np.array(image)
    image = tf.expand_dims(image,0)
    image_resized = tf.image.resize(image,(224,224),method='bicubic') # resolution(늘리거나 작게 했을 때 픽셀값을 조정해주는 것)
    return norm_layer(image_resized).numpy()

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
train_gen = train_datagen.flow_from_dataframe( # 연산속도, fitting 면에서 이득 batch_size가 32로 들어감
    dataframe=train_df,
    x_col='img_path',
    y_col='label',
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    target_size=(224,224)
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
val_gen = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='img_path',
    y_col='label',
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    target_size=(224,224)
)

Found 5992 validated image filenames belonging to 4 classes.
Found 1199 validated image filenames belonging to 4 classes.


# 4. Modeling_DenseNet

In [ ]:
from tensorflow.keras.optimizers import Adam, RMSprop, AdamW

DenseNet121

In [ ]:
# model_Dense1 = DenseNet121(
#     input_shape=(224,224,3),
#     include_top= False,   # 학습할 때 False로
#     weights="imagenet",
# )
# # model_Dense1.summary()  # include_top = True
# x_1 = GlobalAveragePooling2D()(model_Dense1.output)
# outputs_1 = Dense(4, activation='softmax')(x_1)

# model_Dense121 = Model(inputs=model_Dense1.input, outputs=outputs_1)

# DenseNet121 모델 정의 및 레이어 이름 변경
def create_densenet_model_with_custom_names(input_shape, prefix):
    base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=input_shape)

    # 레이어 이름에 접두사 추가
    for layer in base_model.layers:
        layer._name = prefix + layer.name

    x = GlobalAveragePooling2D(name=prefix + 'global_avg_pool')(base_model.output)
    outputs = Dense(4, activation='softmax', name=prefix + 'dense')(x)
    model = Model(inputs=base_model.input, outputs=outputs, name=prefix + 'densenet121')
    return model

# 모델 생성
input_shape = (224, 224, 3)
model_Dense121 = create_densenet_model_with_custom_names(input_shape, 'dense121_')

# 모델 요약
model_Dense121.summary()

In [ ]:
model_Dense121.compile(optimizer=Adam(learning_rate=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
checkpoint_callback = ModelCheckpoint(
    filepath='model-{epoch:02d}-{val_loss:.2f}.keras',
    monitor="val_accuracy",
    save_best_only=True,
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-8
)

callbacks = [
    checkpoint_callback,
    reduce_lr
]

In [ ]:
history_D121 = model_Dense121.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    callbacks=callbacks
)

# model_Dense121.save('/content/drive/MyDrive/DenseNet121.h5')

DenseNet169

In [ ]:
# model_Dense2 = DenseNet169(
#     input_shape=(224,224,3),
#     include_top= False,
#     weights="imagenet",
# )
# # model_Dense2.summary()
# x_2 = GlobalAveragePooling2D()(model_Dense2.output)
# outputs_2 = Dense(4, activation='softmax')(x_2)

# model_Dense169 = Model(inputs=model_Dense2.input, outputs=outputs_2)



# DenseNet169 모델 정의 및 레이어 이름 변경
def create_densenet_model_with_custom_names(input_shape, prefix):
    base_model = DenseNet169(weights="imagenet", include_top=False, input_shape=input_shape)

    # 레이어 이름에 접두사 추가
    for layer in base_model.layers:
        layer._name = prefix + layer.name

    x = GlobalAveragePooling2D(name=prefix + 'global_avg_pool')(base_model.output)
    outputs = Dense(4, activation='softmax', name=prefix + 'dense')(x)
    model = Model(inputs=base_model.input, outputs=outputs, name=prefix + 'densenet169')
    return model

# 모델 생성
input_shape = (224, 224, 3)
model_Dense169 = create_densenet_model_with_custom_names(input_shape, 'dense169_')

# 모델 요약
model_Dense169.summary()

In [ ]:
model_Dense169.compile(optimizer=Adam(learning_rate=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
checkpoint_callback = ModelCheckpoint(
    filepath='model-{epoch:02d}-{val_loss:.2f}.keras',
    monitor="val_accuracy",
    save_best_only=True,
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-8
)

callbacks = [
    checkpoint_callback,
    reduce_lr
]

In [ ]:
history_D169 = model_Dense169.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    callbacks=callbacks
)

In [ ]:
# model_Dense169.save('/content/drive/MyDrive/DenseNet169.h5')

DenseNet201

In [ ]:
# model_Dense3 = DenseNet201(
#     input_shape=(224,224,3),
#     include_top=False,
#     weights="imagenet",
# )
# # model_Dense3.summary()
# x_3 = GlobalAveragePooling2D()(model_Dense3.output)
# outputs_3 = Dense(4, activation='softmax')(x_3)

# model_Dense201 = Model(inputs=model_Dense3.input, outputs=outputs_3)


# DenseNet201 모델 정의 및 레이어 이름 변경
def create_densenet_model_with_custom_names(input_shape, prefix):
    base_model = DenseNet201(weights="imagenet", include_top=False, input_shape=input_shape)

    # 레이어 이름에 접두사 추가
    for layer in base_model.layers:
        layer._name = prefix + layer.name

    x = GlobalAveragePooling2D(name=prefix + 'global_avg_pool')(base_model.output)
    outputs = Dense(4, activation='softmax', name=prefix + 'dense')(x)
    model = Model(inputs=base_model.input, outputs=outputs, name=prefix + 'densenet201')
    return model

# 모델 생성
input_shape = (224, 224, 3)
model_Dense201 = create_densenet_model_with_custom_names(input_shape, 'dense201_')

# 모델 요약
model_Dense201.summary()

In [ ]:
model_Dense201.compile(optimizer=Adam(learning_rate=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
checkpoint_callback = ModelCheckpoint(
    filepath='model-{epoch:02d}-{val_loss:.2f}.keras',
    monitor="val_accuracy",
    save_best_only=True,
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-8
)

callbacks = [
    checkpoint_callback,
    reduce_lr
]

In [ ]:
history_D201 = model_Dense201.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    callbacks=callbacks
)

In [ ]:
# model_Dense201.save('/content/drive/MyDrive/DenseNet201.h5')